In [5]:
import os
import numpy
import json
from collections import Counter
from services.post_processing import post_processing
from services.distance_scoring import distance_scoring
post_process = post_processing()
scoring = distance_scoring()

In [6]:
def round_argv(div, precision, *argv):
    return [round(arg/div,precision) for arg in argv]

In [15]:
def read_from_log(file, target_wav, post_process_method=None):
    sum_init = numpy.zeros((len(target_wav)))
    sum_vow = numpy.zeros((len(target_wav)))
    sum_final = numpy.zeros((len(target_wav)))
    count = numpy.zeros((len(target_wav)))
    target_wav_idx = 0
    target = False
    oracle = []

    with open(file) as f:
        lines = [ln.strip() for ln in f.readlines()]
        
    for line in lines:
        if(target_wav_idx >= len(target_wav)):
            break
            
        if("target audio" in line):
            wav = line.split('/')[-1].split('.')[0]
            if(wav >= target_wav[target_wav_idx][0] and wav <= target_wav[target_wav_idx][1]):
#                 print(line)
                target = True
                count[target_wav_idx] += 1

        elif(target):
            if('oracle' in line):
                oracle = line.split(': ')[1].split(' ')
                # reset state
                initc, vow, finalc = Counter(), Counter(), Counter()                
            elif('predict' in line):
                result = json.loads(line[len('predict: '):].replace("'", '"'))
                post_result = post_process(result['transcript'].split(' '), method=post_process_method,\
                                   target=oracle)
#                 print(result['transcript'])
#                 print(post_result)
#                 print(oracle)
#                 print()
                t_init, t_vow, t_final = post_result
                for x in t_init:
                    initc[x] += 1
                for x in t_vow:
                    vow[x] += 1
                for x in t_final:
                    finalc[x] += 1
            if('==' in line):
                # scoring
                init_score, vow_score, final_score = scoring( (initc, vow, finalc), oracle )
                sum_init[target_wav_idx] += init_score
                sum_vow[target_wav_idx] += vow_score
                sum_final[target_wav_idx] += final_score
                target = False
                if(wav == target_wav[target_wav_idx][1]):
                    target_wav_idx += 1

    return sum_init, sum_vow, sum_final, count
#     print('{} {} {}'.format(sum_init/count, sum_vow/count, sum_final/count))

In [16]:
def get_overall_score(method=None):
    init,vow,final,count = 0,0,0,0

    target_speaker = ['nick', 'bone']
    tinit, tvow, tfinal, tcount = read_from_log('/home/burin/aphasia_backend/decode.log', \
                                        target_wav=[("0006", "0029"), ("0048", "0071")], \
                                        post_process_method=method)
    init += sum(tinit); vow += sum(tvow); final += sum(tfinal); count += sum(tcount);
    for i in range(len(tinit)):
        print("{} init:{} vow:{} final:{}".format(target_speaker[i], *round_argv(tcount[i], 2,tinit[i], tvow[i], tfinal[i])))


    target_speaker = ['me']
    tinit, tvow, tfinal, tcount = read_from_log('/home/burin/aphasia_backend/decode.2.log', \
                                        target_wav=[("0000", "0023")], \
                                        post_process_method=method)
    init += sum(tinit); vow += sum(tvow); final += sum(tfinal); count += sum(tcount);

    for i in range(len(tinit)):
        print("{} init:{} vow:{} final:{}".format(target_speaker[i], *round_argv(tcount[i], 2,tinit[i], tvow[i], tfinal[i])))

    print("Overall: init:{} vow:{} final:{}".format(*round_argv(count, 2, init, vow, final)))

In [17]:
get_overall_score()

nick init:0.52 vow:0.59 final:0.65
bone init:0.49 vow:0.67 final:0.6
me init:0.61 vow:0.69 final:0.69
Overall: init:0.54 vow:0.65 final:0.64


In [6]:
get_overall_score(method='LTB')

nick init:0.58 vow:0.6 final:0.65
bone init:0.53 vow:0.71 final:0.63
me init:0.69 vow:0.7 final:0.65
Overall: init:0.6 vow:0.67 final:0.64


In [7]:
get_overall_score(method='BSS')

nick init:0.57 vow:0.63 final:0.57
bone init:0.5 vow:0.69 final:0.55
me init:0.66 vow:0.7 final:0.7
Overall: init:0.58 vow:0.67 final:0.6


In [5]:
get_overall_score(method='BestMatch')
# exact_match_val = [0,5,3,2] 
# type_match_val = [0,3,1,1]

nick init:0.6 vow:0.7 final:0.78
bone init:0.53 vow:0.74 final:0.69
me init:0.69 vow:0.78 final:0.79
Overall: init:0.61 vow:0.74 final:0.75


In [7]:
get_overall_score(method='BestMatch')
# exact_match_val = [0,4,3,2]
# type_match_val = [0,1,1,1]

nick init:0.6 vow:0.7 final:0.78
bone init:0.56 vow:0.74 final:0.68
me init:0.71 vow:0.77 final:0.8
Overall: init:0.62 vow:0.74 final:0.76


In [8]:
get_overall_score(method='BestMatch')
# exact_match_val = [0,4,3,2]
# type_match_val = [0,2,1,1]

nick init:0.6 vow:0.7 final:0.78
bone init:0.53 vow:0.74 final:0.69
me init:0.69 vow:0.78 final:0.79
Overall: init:0.61 vow:0.74 final:0.75


In [5]:
# exact_match_val = [0,3,3,2] 
# type_match_val = [0,1,1,1]
get_overall_score(method='BestMatch')

nick init:0.6 vow:0.7 final:0.78
bone init:0.56 vow:0.74 final:0.68
me init:0.7 vow:0.78 final:0.8
Overall: init:0.62 vow:0.74 final:0.76


In [11]:
get_overall_score(method='BestMatch')

target audio: /home/burin/aphasia_backend/cache/wav/0006.wav
k^ w aa th
(['w'], ['aa'], [])
['p', 'aa', 'k^']

w aa th
(['w'], ['aa'], [])
['p', 'aa', 'k^']

uu w aa th ii
(['w'], ['aa'], [])
['p', 'aa', 'k^']

a w aa th
(['w'], ['aa'], [])
['p', 'aa', 'k^']

k^ w aa th ii
(['w'], ['aa'], [])
['p', 'aa', 'k^']

w aa th ii
(['w'], ['aa'], [])
['p', 'aa', 'k^']

u w aa th
(['w'], ['aa'], [])
['p', 'aa', 'k^']

d w aa th
(['w'], ['aa'], [])
['p', 'aa', 'k^']

m uu w aa th
(['w'], ['aa'], [])
['p', 'aa', 'k^']

uu w aa s
(['w'], ['aa'], [])
['p', 'aa', 'k^']

a w aa th ii
(['w'], ['aa'], [])
['p', 'aa', 'k^']

ng w aa th
(['w'], ['aa'], [])
['p', 'aa', 'k^']

u w aa th ii
(['w'], ['aa'], [])
['p', 'aa', 'k^']

uu w aa ch
(['w'], ['aa'], [])
['p', 'aa', 'k^']

k^ w aa s
(['w'], ['aa'], [])
['p', 'aa', 'k^']

w aa s
(['w'], ['aa'], [])
['p', 'aa', 'k^']

d w aa th ii
(['w'], ['aa'], [])
['p', 'aa', 'k^']

m uu w aa th ii
(['w'], ['aa'], [])
['p', 'aa', 'k^']

uu w aa s ii
(['w'], ['aa'], [])


l ii vva w w^
(['l'], ['ii'], [])
['l', 'iia']

l vva
(['l'], ['vva'], [])
['l', 'iia']

l iia @@
(['l'], ['iia'], [])
['l', 'iia']

l iia vva k^
(['l'], ['iia'], [])
['l', 'iia']

l iia vva w o
(['l'], ['iia'], [])
['l', 'iia']

l ii vva w k^
(['l'], ['ii'], [])
['l', 'iia']

l vva w w^
(['l'], ['vva'], [])
['l', 'iia']

l ii vva w @@
(['l'], ['ii'], [])
['l', 'iia']

l k^ vva w w^
(['l'], ['vva'], ['k^'])
['l', 'iia']

l l iia vva
(['l'], ['iia'], [])
['l', 'iia']

l ii vva w o
(['l'], ['ii'], [])
['l', 'iia']

l vva w k^
(['l'], ['vva'], [])
['l', 'iia']

l l iia ng^
(['l'], ['iia'], [])
['l', 'iia']

target audio: /home/burin/aphasia_backend/cache/wav/0021.wav
j m uu k^
(['m'], ['uu'], [])
['r', 'uu']

l m uu k^
(['m'], ['uu'], [])
['r', 'uu']

m uu k^ k^
(['m'], ['uu'], [])
['r', 'uu']

b uu k^
(['b'], ['uu'], [])
['r', 'uu']

m uu k^ t^
(['m'], ['uu'], [])
['r', 'uu']

m uu
(['m'], ['uu'], [])
['r', 'uu']

m uu k^ n^
(['m'], ['uu'], [])
['r', 'uu']

m uu k^ d
(['m'], ['uu'], [])

['pr', 'oo', 't^']

j k @@ oo t^
(['k'], ['oo', '@@'], ['t^'])
['pr', 'oo', 't^']

j k uua oo
(['k'], ['uua', 'oo'], [])
['pr', 'oo', 't^']

j k oo oo
(['k'], ['oo'], [])
['pr', 'oo', 't^']

j k uua oo oo t^
(['k'], ['uua', 'oo'], ['t^'])
['pr', 'oo', 't^']

j k pr oo t^
(['pr'], ['oo'], ['t^'])
['pr', 'oo', 't^']

j pr oo t^
(['pr'], ['oo'], ['t^'])
['pr', 'oo', 't^']

j k o oo @@
(['k'], ['o', 'oo'], [])
['pr', 'oo', 't^']

j r oo @@
(['r'], ['oo'], [])
['pr', 'oo', 't^']

j k oo @@
(['k'], ['oo'], [])
['pr', 'oo', 't^']

j k @@ oo
(['k'], ['oo', '@@'], [])
['pr', 'oo', 't^']

j k @@ oo oo t^
(['k'], ['oo', '@@'], ['t^'])
['pr', 'oo', 't^']

j k uua oo oo
(['k'], ['uua', 'oo'], [])
['pr', 'oo', 't^']

j k o oo o
(['k'], ['o', 'oo'], [])
['pr', 'oo', 't^']

j k pr oo
(['pr'], ['oo'], [])
['pr', 'oo', 't^']

j pr oo
(['pr'], ['oo'], [])
['pr', 'oo', 't^']

target audio: /home/burin/aphasia_backend/cache/wav/0029.wav
j k oo
(['k'], ['oo'], [])
['kr', 'aa', 'p^']

j th ii k oo
(['k'], ['

j s
(['s'], [], [])
['f', 'o', 'n^']

s oo n s
(['s'], ['oo'], [])
['f', 'o', 'n^']

s oo p^
(['s'], ['oo'], ['p^'])
['f', 'o', 'n^']

th s
(['s'], [], [])
['f', 'o', 'n^']

m s
(['s'], [], [])
['f', 'o', 'n^']

ch s p^
([], [], ['p^'])
['f', 'o', 'n^']

c s
(['s'], [], [])
['f', 'o', 'n^']

n s
(['s'], [], [])
['f', 'o', 'n^']

f s
(['f'], [], [])
['f', 'o', 'n^']

t s
(['s'], [], [])
['f', 'o', 'n^']

ph s
(['s'], [], [])
['f', 'o', 'n^']

s oo t^ s
(['s'], ['oo'], ['t^'])
['f', 'o', 'n^']

s oo l s
(['s'], ['oo'], [])
['f', 'o', 'n^']

s s p^
([], [], ['p^'])
['f', 'o', 'n^']

s oo u s
(['s'], ['oo'], [])
['f', 'o', 'n^']

s oo n p^
(['n', 's'], ['oo'], ['p^'])
['f', 'o', 'n^']

k s
(['s'], [], [])
['f', 'o', 'n^']

ch s t^
([], [], ['t^'])
['f', 'o', 'n^']

ch s ii
(['s'], ['ii'], [])
['f', 'o', 'n^']

s oo ng p^
(['ng', 's'], ['oo'], ['p^'])
['f', 'o', 'n^']

p s
(['s'], [], [])
['f', 'o', 'n^']

ch s i
(['s'], ['i'], [])
['f', 'o', 'n^']

s oo r s
(['s'], ['oo'], [])
['f', 'o', '

['ch', 'uua', 'j^']

ch uu a j^ k
(['ch'], ['uu', 'a'], ['j^'])
['ch', 'uua', 'j^']

t^ ch uua a j^ c
(['ch'], ['uua', 'a'], ['j^'])
['ch', 'uua', 'j^']

ch uu w a j^ t
(['w', 'ch'], ['uu', 'a'], ['j^'])
['ch', 'uua', 'j^']

ch uua a j^ f j^
(['ch'], ['uua', 'a'], ['j^'])
['ch', 'uua', 'j^']

ch uu w a j^ th
(['w', 'ch'], ['uu', 'a'], ['j^'])
['ch', 'uua', 'j^']

p^ ch uua a j^ f
(['ch'], ['uua', 'a'], ['j^'])
['ch', 'uua', 'j^']

p^ ch uua a j^ pr
(['ch'], ['uua', 'a'], ['j^'])
['ch', 'uua', 'j^']

target audio: /home/burin/aphasia_backend/cache/wav/0008.wav
m ng aa s j^
(['ng', 's'], ['aa'], ['j^'])
['b', 'a', 'j^']

m ng aa ch j^
(['ng', 'ch'], ['aa'], ['j^'])
['b', 'a', 'j^']

m ng aa j^ s j^
(['ng'], ['aa'], ['j^'])
['b', 'a', 'j^']

m ng aa i j^
(['ng'], ['aa', 'i'], ['j^'])
['b', 'a', 'j^']

m ng aa e j^
(['ng'], ['aa', 'e'], ['j^'])
['b', 'a', 'j^']

m ng aa j j^
(['ng', 'j'], ['aa'], ['j^'])
['b', 'a', 'j^']

m ng aa j^ k^ j^
(['ng'], ['aa'], ['j^'])
['b', 'a', 'j^']

m ng aa 

(['j'], ['iia'], [])
['j', 'aa', 'w^']

m iia xx h aa o ng^
(['h'], ['aa', 'o'], ['ng^'])
['j', 'aa', 'w^']

j iia xx h aa o ng^
(['j'], ['iia'], [])
['j', 'aa', 'w^']

target audio: /home/burin/aphasia_backend/cache/wav/0022.wav
k @@ m oo n^ k^
(['m'], ['oo'], ['n^'])
['pr', 'oo', 't^']

k s m oo n^ k^
(['m'], ['oo'], ['n^'])
['pr', 'oo', 't^']

s m oo n^ k^
(['m'], ['oo'], ['n^'])
['pr', 'oo', 't^']

aa m oo n^ k^
(['m'], ['oo'], ['n^'])
['pr', 'oo', 't^']

pr m oo n^ k^
(['pr', 'm'], ['oo'], ['n^'])
['pr', 'oo', 't^']

k m oo ng^ n^ k^
(['m'], ['oo'], ['ng^'])
['pr', 'oo', 't^']

k @@ m oo ng^ n^ k^
(['m'], ['oo'], ['ng^'])
['pr', 'oo', 't^']

k s m oo ng^ n^ k^
(['m'], ['oo'], ['ng^'])
['pr', 'oo', 't^']

k t^ m oo n^ k^
(['m'], ['oo'], ['n^'])
['pr', 'oo', 't^']

k pr m oo n^ k^
(['pr', 'm'], ['oo'], ['n^'])
['pr', 'oo', 't^']

t vva m oo n^ k^
(['m'], ['oo'], ['n^'])
['pr', 'oo', 't^']

s m oo ng^ n^ k^
(['m'], ['oo'], ['ng^'])
['pr', 'oo', 't^']

aa m oo ng^ n^ k^
(['m'], ['oo']